In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
data_path = "../data/raw/ventas_empresa.csv"

df = pd.read_csv(data_path)

df.head()


,fecha,cliente_id,producto,categoria,precio,costo,canal,region,descuento,ingreso,ganancia
0,2024-01-01 00:00:00,1117,Cafetera automática,Hogar,159.986158,103.189022,Marketplace,Sur,0.186831,130.095761,26.906739
1,2024-01-01 01:00:00,1931,Mesa de centro,Hogar,231.584256,129.209217,Web,Norte,0.042490,221.744238,92.535021
2,2024-01-01 02:00:00,1602,Playera deportiva Nike,Ropa,32.966927,14.212245,Web,Sur,0.061623,30.935401,16.723156
3,2024-01-01 03:00:00,1339,Laptop Lenovo IdeaPad,Electrónica,819.356982,404.335988,Web,Occidente,0.107733,731.085310,326.749322
4,2024-01-01 04:00:00,1800,Laptop Lenovo IdeaPad,Electrónica,604.871535,530.172061,App móvil,Occidente,0.082860,554.752171,24.580110


In [3]:
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       5000 non-null   object 
 1   cliente_id  5000 non-null   int64  
 2   producto    5000 non-null   object 
 3   categoria   5000 non-null   object 
 4   precio      5000 non-null   float64
 5   costo       5000 non-null   float64
 6   canal       5000 non-null   object 
 7   region      5000 non-null   object 
 8   descuento   5000 non-null   float64
 9   ingreso     5000 non-null   float64
 10  ganancia    5000 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 429.8+ KB


fecha         0
cliente_id    0
producto      0
categoria     0
precio        0
costo         0
canal         0
region        0
descuento     0
ingreso       0
ganancia      0
dtype: int64

In [4]:
df["fecha"] = pd.to_datetime(df["fecha"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       5000 non-null   datetime64[ns]
 1   cliente_id  5000 non-null   int64         
 2   producto    5000 non-null   object        
 3   categoria   5000 non-null   object        
 4   precio      5000 non-null   float64       
 5   costo       5000 non-null   float64       
 6   canal       5000 non-null   object        
 7   region      5000 non-null   object        
 8   descuento   5000 non-null   float64       
 9   ingreso     5000 non-null   float64       
 10  ganancia    5000 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(4)
memory usage: 429.8+ KB


In [5]:
df = df[df["precio"] > 0]
df = df[df["costo"] >= 0]
df = df[df["precio"] >= df["costo"]]


In [6]:
df["margen_pct"] = df["ganancia"] / df["ingreso"]


In [7]:
ticket_cliente = (
    df.groupby("cliente_id")["ingreso"]
    .sum()
    .reset_index(name="ticket_cliente")
)

df = df.merge(ticket_cliente, on="cliente_id", how="left")


In [8]:
df["mes"] = df["fecha"].dt.month
df["anio"] = df["fecha"].dt.year


In [9]:
df["segmento_cliente"] = pd.qcut(
    df["ticket_cliente"],
    q=3,
    labels=["Bajo", "Medio", "Alto"]
)


In [10]:
OUTPUT_PATH = Path("../data/processed/ventas_limpias.csv")

df.to_csv(OUTPUT_PATH, index=False)

print("Datos limpios guardados ✅")


Datos limpios guardados ✅
